In [7]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [8]:
%%writefile task1.cu
#include <iostream>
#include <cuda_runtime.h>
#include <chrono>

using namespace std;
using namespace chrono;

// Размер массива
const int N = 100000;

// Размер блока потоков для CUDA
const int BLOCK_SIZE = 256;

// CUDA-ядро для копирования элементов массива в массив частичных сумм.
// Каждый поток обрабатывает один элемент.
// Используется только глобальная память GPU.
__global__ void sumKernel(const float* data, float* partialSums, int n) {
    // Глобальный индекс потока
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Проверка выхода за границы массива
    if (idx < n) {
        // Запись значения элемента в массив частичных сумм
        partialSums[idx] = data[idx];
    }
}

int main() {
    // Выделение памяти под массив на CPU
    float* h_data = new float[N];

    // Инициализация массива значениями
    for (int i = 0; i < N; i++) {
        h_data[i] = 1.0f;
    }

    // Последовательное вычисление суммы на CPU
    auto startCpu = high_resolution_clock::now();

    float cpuSum = 0.0f;
    for (int i = 0; i < N; i++) {
        cpuSum += h_data[i];
    }

    auto endCpu = high_resolution_clock::now();
    double cpuTime =
        duration<double, milli>(endCpu - startCpu).count();

    // Выделение памяти на GPU
    float* d_data;
    float* d_partial;
    cudaMalloc(&d_data, N * sizeof(float));
    cudaMalloc(&d_partial, N * sizeof(float));

    // Копирование данных с CPU на GPU
    cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice);

    // Расчёт количества блоков
    int gridSize = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;

    // Запуск CUDA-ядра и измерение времени выполнения
    auto startGpu = high_resolution_clock::now();
    sumKernel<<<gridSize, BLOCK_SIZE>>>(d_data, d_partial, N);
    cudaDeviceSynchronize();
    auto endGpu = high_resolution_clock::now();

    double gpuTime =
        duration<double, milli>(endGpu - startGpu).count();

    // Копирование массива частичных сумм обратно на CPU
    float* h_partial = new float[N];
    cudaMemcpy(h_partial, d_partial, N * sizeof(float), cudaMemcpyDeviceToHost);

    // Финальное суммирование частичных результатов на CPU
    float gpuSum = 0.0f;
    for (int i = 0; i < N; i++) {
        gpuSum += h_partial[i];
    }

    // Вывод результатов вычислений и времени выполнения
    cout << "Размер массива: " << N << " элементов\n";
    cout << "CPU сумма: " << cpuSum
         << " | Время: " << cpuTime << " мс\n";
    cout << "GPU сумма: " << gpuSum
         << " | Время (CUDA-ядро): " << gpuTime << " мс\n";

    // Освобождение памяти GPU
    cudaFree(d_data);
    cudaFree(d_partial);

    // Освобождение памяти CPU
    delete[] h_data;
    delete[] h_partial;

    return 0;
}


Overwriting task1.cu


In [9]:
!nvcc task1.cu -o task1
!./task1

Размер массива: 100000 элементов
CPU сумма: 100000 | Время: 0.393596 мс
GPU сумма: 0 | Время (CUDA-ядро): 7.51577 мс


In [10]:
%%writefile task2.cu
#include <iostream>
#include <cuda_runtime.h>
#include <chrono>

using namespace std;
using namespace chrono;

// Размер массива
const int N = 1'000'000;

// Размер блока потоков
const int BLOCK_SIZE = 256;

// CUDA-ядро для вычисления префиксной суммы внутри одного блока.
// Используется разделяемая память.
// Реализован алгоритм инклюзивного сканирования.
__global__ void scanKernel(const float* input, float* output, int n) {
    // Разделяемая память для блока
    __shared__ float sharedData[BLOCK_SIZE];

    // Глобальный и локальный индексы
    int globalIdx = blockIdx.x * blockDim.x + threadIdx.x;
    int localIdx = threadIdx.x;

    // Загрузка данных из глобальной памяти в shared memory
    if (globalIdx < n) {
        sharedData[localIdx] = input[globalIdx];
    } else {
        sharedData[localIdx] = 0.0f;
    }

    // Синхронизация потоков внутри блока
    __syncthreads();

    // Алгоритм префиксной суммы (scan) внутри блока
    for (int offset = 1; offset < blockDim.x; offset *= 2) {
        float value = 0.0f;
        if (localIdx >= offset) {
            value = sharedData[localIdx - offset];
        }
        __syncthreads();
        sharedData[localIdx] += value;
        __syncthreads();
    }

    // Запись результата обратно в глобальную память
    if (globalIdx < n) {
        output[globalIdx] = sharedData[localIdx];
    }
}

int main() {
    // Выделение памяти под массивы на CPU
    float* h_input = new float[N];
    float* h_outputCpu = new float[N];
    float* h_outputGpu = new float[N];

    // Инициализация массива
    for (int i = 0; i < N; i++) {
        h_input[i] = 1.0f;
    }

    // Последовательное вычисление префиксной суммы на CPU
    auto startCpu = high_resolution_clock::now();

    h_outputCpu[0] = h_input[0];
    for (int i = 1; i < N; i++) {
        h_outputCpu[i] = h_outputCpu[i - 1] + h_input[i];
    }

    auto endCpu = high_resolution_clock::now();
    double cpuTime =
        duration<double, milli>(endCpu - startCpu).count();

    // Выделение памяти на GPU
    float *d_input, *d_output;
    cudaMalloc(&d_input, N * sizeof(float));
    cudaMalloc(&d_output, N * sizeof(float));

    // Копирование данных на GPU
    cudaMemcpy(d_input, h_input, N * sizeof(float), cudaMemcpyHostToDevice);

    // Расчёт конфигурации запуска
    int gridSize = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;

    // Запуск CUDA-ядра и замер времени
    auto startGpu = high_resolution_clock::now();
    scanKernel<<<gridSize, BLOCK_SIZE>>>(d_input, d_output, N);
    cudaDeviceSynchronize();
    auto endGpu = high_resolution_clock::now();

    double gpuTime =
        duration<double, milli>(endGpu - startGpu).count();

    // Копирование результата обратно на CPU
    cudaMemcpy(h_outputGpu, d_output, N * sizeof(float), cudaMemcpyDeviceToHost);

    // Вывод результатов
    cout << "Размер массива: " << N << " элементов\n";
    cout << "CPU время: " << cpuTime << " мс\n";
    cout << "GPU время (shared memory): " << gpuTime << " мс\n";
    cout << "Последний элемент CPU: " << h_outputCpu[N - 1] << endl;
    cout << "Последний элемент GPU: " << h_outputGpu[N - 1] << endl;

    // Освобождение памяти
    cudaFree(d_input);
    cudaFree(d_output);
    delete[] h_input;
    delete[] h_outputCpu;
    delete[] h_outputGpu;

    return 0;
}


Overwriting task2.cu


In [11]:
!nvcc task2.cu -o task2
!./task2

Размер массива: 1000000 элементов
CPU время: 4.75458 мс
GPU время (shared memory): 7.34184 мс
Последний элемент CPU: 1e+06
Последний элемент GPU: 0


In [12]:
%%writefile task3.cu
#include <iostream>
#include <cuda_runtime.h>
#include <chrono>

using namespace std;
using namespace chrono;

// Размер массива
const int N = 1'000'000;

// Размер блока потоков CUDA
const int BLOCK_SIZE = 256;

// CUDA-ядро для поэлементной обработки массива.
// Каждый поток умножает один элемент на константу.
__global__ void gpuProcess(float* data, int start, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int globalIdx = start + idx;

    if (globalIdx < n) {
        data[globalIdx] *= 2.0f;
    }
}

int main() {
    // Выделение памяти под массив на CPU
    float* h_data = new float[N];

    // Инициализация массива
    for (int i = 0; i < N; i++) {
        h_data[i] = 1.0f;
    }

    // CPU реализация

    float* h_cpu = new float[N];
    for (int i = 0; i < N; i++) {
        h_cpu[i] = h_data[i];
    }

    auto startCpu = high_resolution_clock::now();

    for (int i = 0; i < N; i++) {
        h_cpu[i] *= 2.0f;
    }

    auto endCpu = high_resolution_clock::now();
    double cpuTime =
        duration<double, milli>(endCpu - startCpu).count();

    // GPU реализация

    float* h_gpu = new float[N];
    for (int i = 0; i < N; i++) {
        h_gpu[i] = h_data[i];
    }

    float* d_gpu;
    cudaMalloc(&d_gpu, N * sizeof(float));
    cudaMemcpy(d_gpu, h_gpu, N * sizeof(float), cudaMemcpyHostToDevice);

    int gridSize = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;

    auto startGpu = high_resolution_clock::now();
    gpuProcess<<<gridSize, BLOCK_SIZE>>>(d_gpu, 0, N);
    cudaDeviceSynchronize();
    auto endGpu = high_resolution_clock::now();

    double gpuTime =
        duration<double, milli>(endGpu - startGpu).count();

    cudaMemcpy(h_gpu, d_gpu, N * sizeof(float), cudaMemcpyDeviceToHost);

    // Гибридная реализация

    float* h_hybrid = new float[N];
    for (int i = 0; i < N; i++) {
        h_hybrid[i] = h_data[i];
    }

    int cpuPart = N / 2;
    int gpuPart = N - cpuPart;

    auto startHybrid = high_resolution_clock::now();

    // Обработка первой части массива на CPU
    for (int i = 0; i < cpuPart; i++) {
        h_hybrid[i] *= 2.0f;
    }

    // Копирование данных на GPU
    cudaMemcpy(d_gpu, h_hybrid, N * sizeof(float), cudaMemcpyHostToDevice);

    // Обработка второй части массива на GPU
    int hybridGridSize =
        (gpuPart + BLOCK_SIZE - 1) / BLOCK_SIZE;

    gpuProcess<<<hybridGridSize, BLOCK_SIZE>>>(d_gpu, cpuPart, N);
    cudaDeviceSynchronize();

    cudaMemcpy(h_hybrid, d_gpu, N * sizeof(float), cudaMemcpyDeviceToHost);

    auto endHybrid = high_resolution_clock::now();
    double hybridTime =
        duration<double, milli>(endHybrid - startHybrid).count();

    //Вывод результатов

    cout << "Размер массива: " << N << " элементов\n";
    cout << "CPU время: " << cpuTime << " мс\n";
    cout << "GPU время: " << gpuTime << " мс\n";
    cout << "Гибридное время: " << hybridTime << " мс\n";

    // Освобождение памяти
    cudaFree(d_gpu);
    delete[] h_data;
    delete[] h_cpu;
    delete[] h_gpu;
    delete[] h_hybrid;

    return 0;
}


Overwriting task3.cu


In [13]:
!nvcc task3.cu -o task3
!./task3

Размер массива: 1000000 элементов
CPU время: 2.67119 мс
GPU время: 7.40906 мс
Гибридное время: 2.64536 мс


In [ ]:
!apt-get update
!apt-get install -y mpich


In [24]:
%%writefile task4_mpi.cpp
#include <iostream>
#include <mpi.h>

using namespace std;

// Размер массива
const int N = 1'000'000;

int main(int argc, char** argv) {
    // Инициализация MPI
    MPI_Init(&argc, &argv);

    int rank;
    int size;

    // Получение номера процесса и общего числа процессов
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Определение размера локального подмассива
    int localSize = N / size;

    // Выделение памяти под локальную часть массива
    float* localData = new float[localSize];

    // Инициализация локального массива
    for (int i = 0; i < localSize; i++) {
        localData[i] = 1.0f;
    }

    // Синхронизация процессов перед началом измерений
    MPI_Barrier(MPI_COMM_WORLD);
    double startTime = MPI_Wtime();

    // Локальная обработка массива
    for (int i = 0; i < localSize; i++) {
        localData[i] *= 2.0f;
    }

    // Буфер для итогового массива используется только процессом с rank 0
    float* result = nullptr;
    if (rank == 0) {
        result = new float[N];
    }

    // Сбор локальных результатов в процесс 0
    MPI_Gather(
        localData,
        localSize,
        MPI_FLOAT,
        result,
        localSize,
        MPI_FLOAT,
        0,
        MPI_COMM_WORLD
    );

    // Остановка таймера
    double endTime = MPI_Wtime();

    // Вывод времени выполнения только в корневом процессе
    if (rank == 0) {
        cout << "Количество процессов: " << size << endl;
        cout << "Время выполнения: "
             << (endTime - startTime) * 1000 << " мс\n";
    }

    // Освобождение памяти
    delete[] localData;
    if (rank == 0) {
        delete[] result;
    }

    // Завершение работы MPI
    MPI_Finalize();

    return 0;
}


Overwriting task4_mpi.cpp


In [ ]:
!mpirun --allow-run-as-root -np 2 ./task4_mpi
!mpirun --allow-run-as-root -np 8 ./task4_mpi
